## 아이리스 다중 분류

In [1]:
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
import pandas as pd
df = pd.read_csv('dataset/iris.csv', 
                 names = ["sepal_length", "sepal_width", "petal_length", "petal_width", "species"])
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [4]:
dataset = df.values
x_data = np.array(dataset[:,0:4], dtype=np.float32)
Y_obj = dataset[:,4]
Y_obj[0], Y_obj[50], Y_obj[100]

('Iris-setosa', 'Iris-versicolor', 'Iris-virginica')

In [5]:
Y_obj = pd.get_dummies(Y_obj)
Y_obj.head()

,Iris-setosa,Iris-versicolor,Iris-virginica
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0


In [6]:
type(Y_obj), type(x_data)

(pandas.core.frame.DataFrame, numpy.ndarray)

In [7]:
y_data = np.array(Y_obj.values, dtype=np.float32)
y_data[0], y_data[50], y_data[100]

(array([1., 0., 0.], dtype=float32),
 array([0., 1., 0.], dtype=float32),
 array([0., 0., 1.], dtype=float32))

In [8]:
print(x_data.shape, y_data.shape)

(150, 4) (150, 3)


In [9]:
# seed 값 설정
seed = 0
tf.set_random_seed(seed)

In [10]:
# 입력 값을 플레이스 홀더에 저장
X = tf.placeholder(tf.float32, shape=[None, 4])
Y = tf.placeholder(tf.float32, shape=[None, 3])

In [11]:
# Layer 1
W1 = tf.Variable(tf.random_normal([4,16], dtype=tf.float32)) 
b1 = tf.Variable(tf.random_normal([16], dtype=tf.float32))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)

In [12]:
# Layer 2
W2 = tf.Variable(tf.random_normal([16,3], dtype=tf.float32)) 
b2 = tf.Variable(tf.random_normal([3], dtype=tf.float32))
hypo = tf.nn.softmax(tf.matmul(L1, W2) + b2)

In [13]:
# Categorical Cross Entropy
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypo)))

In [14]:
predicted = tf.equal(tf.argmax(Y, 1), tf.argmax(hypo, 1))
accuracy = tf.reduce_mean(tf.cast(predicted, dtype=tf.float32))

In [15]:
lr = 0.1
train = tf.train.AdamOptimizer(learning_rate=lr).minimize(cost)

In [16]:
# 학습
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(51):
    sess.run(train, feed_dict = {X: x_data, Y: y_data})
    if epoch % 10 == 0:
        cost_val, acc_val = sess.run([cost, accuracy], feed_dict = {X: x_data, Y: y_data})
        print("step=%d, cost_val=%.4f, accuracy: %.4f" % (epoch + 1, cost_val, acc_val))

step=1, cost_val=907.3970, accuracy: 0.3333
step=11, cost_val=51.6845, accuracy: 0.8200
step=21, cost_val=35.2100, accuracy: 0.9000
step=31, cost_val=19.8225, accuracy: 0.9667
step=41, cost_val=16.0508, accuracy: 0.9733
step=51, cost_val=12.9328, accuracy: 0.9800


In [17]:
# 평가
acc = sess.run(accuracy, feed_dict = {X: x_data, Y: y_data})
print("Accuracy: %.4f" % acc)

Accuracy: 0.9800


### 아래와 같은 방법을 써서 해도 됨

In [18]:
# Layer 2
W2 = tf.Variable(tf.random_normal([16,3], dtype=tf.float32)) 
b2 = tf.Variable(tf.random_normal([3], dtype=tf.float32))
hypo = tf.matmul(L1, W2) + b2

In [19]:
# Categorical Cross Entropy
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypo, labels=Y))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [20]:
predicted = tf.equal(tf.argmax(Y, 1), tf.argmax(hypo, 1))
accuracy = tf.reduce_mean(tf.cast(predicted, dtype=tf.float64))

In [21]:
lr = 0.1
train = tf.train.AdamOptimizer(learning_rate=lr).minimize(cost)

In [27]:
# 학습
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(201):
    sess.run(train, feed_dict = {X: x_data, Y: y_data})
    if epoch % 20 == 0:
        cost_val, acc_val = sess.run([cost, accuracy], feed_dict = {X: x_data, Y: y_data})
        print("step=%d, cost_val=%.4f, accuracy: %.4f" % (epoch + 1, cost_val, acc_val))

step=1, cost_val=6.4135, accuracy: 0.0200
step=21, cost_val=0.3781, accuracy: 0.8733
step=41, cost_val=0.1491, accuracy: 0.9667
step=61, cost_val=0.0970, accuracy: 0.9667
step=81, cost_val=0.0772, accuracy: 0.9667
step=101, cost_val=0.0689, accuracy: 0.9667
step=121, cost_val=0.0638, accuracy: 0.9733
step=141, cost_val=0.0602, accuracy: 0.9800
step=161, cost_val=0.0573, accuracy: 0.9800
step=181, cost_val=0.0550, accuracy: 0.9800
step=201, cost_val=0.0531, accuracy: 0.9800


In [28]:
# 평가
acc = sess.run(accuracy, feed_dict = {X: x_data, Y: y_data})
print("Accuracy: %.4f" % acc)

Accuracy: 0.9800
